# Optimizing the design and operation of a DAC system under varying ambient conditions
This case study is based on a model developed here:
[Wiegner, Jan F., Alexa Grimm, Lukas Weimann, and Matteo Gazzani. "Optimal design and operation of solid sorbent direct air capture processes at varying ambient conditions." Industrial & Engineering Chemistry Research 61, no. 34 (2022): 12649-12667.
](https://pubs.acs.org/doi/10.1021/acs.iecr.2c00681)

We want to optimize the design and operation of a DAC with a target net negative emissions of 1000 t in the Netherlands. We therefore load climate data for a location in the Netherlands. We store the CO2 in a not further defined CO2 sink.

## Create templates
We set the input data path and in this directory we can add input data templates for the model configuration and the topology with the function create_optimization_templates.


In [1]:
import json
import src.data_preprocessing as dp
from src.energyhub import EnergyHub
from pathlib import Path

input_data_path = Path("./caseStudies/dac")
dp.create_optimization_templates(input_data_path)

C:\Users\6574114\PycharmProjects\EHUB-Py\src\components\technologies\genericTechnologies\stor.py:16: SyntaxWarning: invalid escape sequence '\l'
  """
C:\Users\6574114\PycharmProjects\EHUB-Py\src\components\technologies\genericTechnologies\sink.py:12: SyntaxWarning: invalid escape sequence '\l'
  """


## Adapt Topology
We need to adapt the topology as well as the model configuration file to our case study. This can be done either in the file itself (Topology.json) or, as we do it here) via some lines of code.
For the topology, we need to change the following:
- Change nodes: nl
- Change carriers: electricity, heat and CO2captured
- Change investment periods: period1
- The options regarding the time frame we can leave at the default (one year with hourly operation)

In [2]:
# Load json template
with open(input_data_path / "Topology.json", "r") as json_file:
    topology = json.load(json_file)
# Nodes
topology["nodes"] = ["nl"]
# Carriers:
topology["carriers"] = ["electricity", "heat", "CO2captured"]
# Investment periods:
topology["investment_periods"] = ["period1"]
# Save json template
with open(input_data_path / "Topology.json", "w") as json_file:
    json.dump(topology, json_file, indent=4)

## Adapt Model Configurations
Now, we need to adapt the model configurations respectively. As the DAC model is rather complex, we also cluster the full resolution into 50 typical days (method 1).
- Change objective to 'costs_emissionlimit' (this minimizes annualized costs at an emission limit)
- Change emission limit to -1000 to account for the emission target
- Change the number of typical days to 50 and select time aggregation method 1 

In [3]:
# Load json template
with open(input_data_path / "ConfigModel.json", "r") as json_file:
    configuration = json.load(json_file)
# Change objective
configuration["optimization"]["objective"]["value"] = "costs_emissionlimit"
# Set emission limit:
configuration["optimization"]["emission_limit"]["value"] = -1000
# Set time aggregation settings:
configuration["optimization"]["typicaldays"]["N"]["value"] = 50
configuration["optimization"]["typicaldays"]["method"]["value"] = 1
# Save json template
with open(input_data_path / "ConfigModel.json", "w") as json_file:
    json.dump(configuration, json_file, indent=4)

## Define input data
We first create all required input data files based on the topology file and then add the DAC technology as a new technology to the respective node.
Additionally we:
- copy over technology data to the input data folder
- define climate data for a dutch location (the location is used by default if nothing in NodeLocation.csv is specified)

In [4]:
dp.create_input_data_folder_template(input_data_path)

# Add DAC as a new technology
with open(input_data_path / "period1" / "node_data" / "nl" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["DAC_Adsorption"]
technologies["existing"] = {"PermanentStorage_CO2_simple": 10000}

with open(input_data_path / "period1" / "node_data" / "nl" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)

# Copy over technology files
dp.copy_technology_data(input_data_path, Path("."))

# Define climate data
dp.load_climate_data_from_api(input_data_path)


Importing Climate Data...
Importing Climate Data successful


## Run model - infeasibility
Now, we have defined all required data to run the model. It will be infeasible though...

In [5]:
m = EnergyHub()
m.read_data(input_data_path)
m.quick_solve()

--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Topology read successfully
Model Configuration read successfully
Found NaN values in data for investment period period1, node nl, key1 CarbonCost, carrier global, key2 price. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 CarbonCost, carrier global, key2 subsidy. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 ClimateData, carrier global, key2 TECHNOLOGYNAME_hydro_inflow. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 CarrierData, carrier electricity, key2 Demand. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 CarrierData, carrier electricity, key2 Import limit. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 CarrierData, carrier electricity, key2 Export limit. Replaced with zero

C:\Users\6574114\PycharmProjects\EHUB-Py\venv\Lib\site-packages\tsam\timeseriesaggregation.py:1099: UserWarning: Something went wrong... At least one maximal value of the aggregated time series exceeds the maximal value the input time series for: [('tec_series', 'nl', 'DAC_Adsorption', 'a', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'a', 1), ('tec_series', 'nl', 'DAC_Adsorption', 'a', 2), ('tec_series', 'nl', 'DAC_Adsorption', 'alpha', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'b', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'b', 2), ('tec_series', 'nl', 'DAC_Adsorption', 'beta', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'beta', 1), ('tec_series', 'nl', 'DAC_Adsorption', 'delta', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'delta', 1), ('tec_series', 'nl', 'DAC_Adsorption', 'el_in_max', ''), ('tec_series', 'nl', 'DAC_Adsorption', 'gamma', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'gamma', 1), ('tec_series', 'nl', 'DAC_Adsorption', 'out_max', ''), ('tec_series', 'nl', 'DAC_Adsorption',

Clustered data successfully
--- Reading in data complete ---
--- Constructing Model ---
Constructing Investment Period period1
	 - Adding Technology DAC_Adsorption
			gdp.bigm Transformation...
	 - Adding Technology PermanentStorage_CO2_simple
Constructing model completed in 4s
Constructing balances...
Constructing balances completed in 1s
Defined constraint on net emissions
Set objective on cost
____________________________________________________________
Solving Model...
Set parameter LogFile to value "userData\20240606183031-1\log.txt"
Solver log file: userData\20240606183031-1\log.txt
Set parameter TimeLimit to value 36000
Set parameter MIPGap to value 0.001
Set parameter NodefileStart to value 60
Set parameter LPWarmStart to value 0
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize 

## Allowing for heat and electricity import
The model is infeasible, because we did not define where heat or electricity should come from. Here, we allow for electricity and heat import at a certain price at no additional emissions. As such we define:
- An abitrary import limit on heat and electricity (1GW)
- An import price on electricity of 60 EUR/MWh
- An import price on heat of 20 EUR/MWh

In [6]:
dp.fill_carrier_data(input_data_path, value=1000, columns=['Import limit'], carriers=['electricity', 'heat'], nodes=['nl'])
dp.fill_carrier_data(input_data_path, value=60, columns=['Import price'], carriers=['electricity'], nodes=['nl'])
dp.fill_carrier_data(input_data_path, value=20, columns=['Import price'], carriers=['heat'], nodes=['nl'])

## Run model again
Now, the model should be feasible

In [ ]:
m = EnergyHub()
m.read_data(input_data_path)
m.quick_solve()

--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Topology read successfully
Model Configuration read successfully
Found NaN values in data for investment period period1, node nl, key1 CarbonCost, carrier global, key2 price. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 CarbonCost, carrier global, key2 subsidy. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 ClimateData, carrier global, key2 TECHNOLOGYNAME_hydro_inflow. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 CarrierData, carrier electricity, key2 Demand. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 CarrierData, carrier electricity, key2 Export limit. Replaced with zeros.
Found NaN values in data for investment period period1, node nl, key1 CarrierData, carrier electricity, key2 Export price. Replaced with zero

C:\Users\6574114\PycharmProjects\EHUB-Py\venv\Lib\site-packages\tsam\timeseriesaggregation.py:1099: UserWarning: Something went wrong... At least one maximal value of the aggregated time series exceeds the maximal value the input time series for: [('tec_series', 'nl', 'DAC_Adsorption', 'a', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'a', 1), ('tec_series', 'nl', 'DAC_Adsorption', 'b', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'beta', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'beta', 1), ('tec_series', 'nl', 'DAC_Adsorption', 'delta', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'delta', 1), ('tec_series', 'nl', 'DAC_Adsorption', 'gamma', 0), ('tec_series', 'nl', 'DAC_Adsorption', 'out_max', ''), ('time_series', 'nl', 'ClimateData', 'global', 'dhi'), ('time_series', 'nl', 'ClimateData', 'global', 'dni'), ('time_series', 'nl', 'ClimateData', 'global', 'ghi'), ('time_series', 'nl', 'ClimateData', 'global', 'rh'), ('time_series', 'nl', 'ClimateData', 'global', 'ws10')]
  warnings.warn(


Clustered data successfully
--- Reading in data complete ---
--- Constructing Model ---
Constructing Investment Period period1
	 - Adding Technology DAC_Adsorption
			gdp.bigm Transformation...
	 - Adding Technology PermanentStorage_CO2_simple
Constructing model completed in 4s
Constructing balances...
Constructing balances completed in 1s
Defined constraint on net emissions
Set objective on cost
____________________________________________________________
Solving Model...
Set parameter LogFile to value "userData\20240606190231-1\log.txt"
Solver log file: userData\20240606190231-1\log.txt
Set parameter TimeLimit to value 36000
Set parameter MIPGap to value 0.001
Set parameter NodefileStart to value 60
Set parameter LPWarmStart to value 0
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize 

## Visualization
The results can be inspected using the provided [visualization platform](https://resultvisualization.streamlit.app/) for some basic plots. The figures below are screenshots from the visualization platform.

### Costs

### Heat requirements

### Electricity requirements

### DAC operation

